[Quantopian](https://www.quantopian.com/algorithms), which is a platform that allows us to write and back-test Python-powered trading strategies very easily.

What Quantopian does is it adds a GUI layer on top of the Zipline back testing library for Python, along with a bunch of data sources as well, many of which are completely free to work with. You can also get capital allocations from Quantopian by licensing your strategy to them if you meet certain criteria. Generally, a beta between -0.3 and +0.3 is a good starting point, but you also need to have other healthy metrics to compete. More on this later, let's learn about the basics of Quantopian first. Since Quantopian is powered by primarily open sourced libraries like Zipline, Alphalens, and Pyfolio, you can also run a Quantopian-like platform locally if you like. I find most people who are interested in running locally are interested in this to keep their algorithms private. Quantopian does not view your algorithms unless you give them permission to, and the community only sees your algorithms if you share them. I highly encourage you to view your relationship with Quantopian not as an adversarial one, but instead as a partnership. If you come up with something of high quality, Quantopian is very interested in working with, and has the funding to invest in, you. In this relationship, Quantopian is bringing the platform, funding, and other experts in the field to help you, it's a pretty good deal in my opinion.

1. Python Editor - This is where you code your Python logic for the algoirthm.
2. Built-algorithm results - When you build the algorithm, graphical results will apppear here.
3. Log / Error output - Any console output / log information will come here. It's common to have your program output various bits of text for debugging or just for more information.
4. Build Algorithm - Use this to quickly test what you've written. Results wont be saved, but you can see the result in the built-algorithm results section.
5. Full Backtest - This will run a full back test based on your current algorithm. Full back tests come with a bit more analysis, results are saved, and the algorithm that generated those results is also saved, so you can go back through back tests and view the exact code that generated a specific result.

In [ ]:
"""
This is a template algorithm on Quantopian for you to adapt and fill in.
"""
from quantopian.algorithm import attach_pipeline, pipeline_output
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import AverageDollarVolume
 
def initialize(context):
    """
    Called once at the start of the algorithm.
    """   
    # Rebalance every day, 1 hour after market open.
    schedule_function(my_rebalance, date_rules.every_day(), time_rules.market_open(hours=1))
     
    # Record tracking variables at the end of each day.
    schedule_function(my_record_vars, date_rules.every_day(), time_rules.market_close())
     
    # Create our dynamic stock selector.
    attach_pipeline(make_pipeline(), 'my_pipeline')
         
def make_pipeline():
    """
    A function to create our dynamic stock selector (pipeline). Documentation on
    pipeline can be found here: https://www.quantopian.com/help#pipeline-title
    """
    
     
    # Create a dollar volume factor.
    dollar_volume = AverageDollarVolume(window_length=1)
 
    # Pick the top 1% of stocks ranked by dollar volume.
    high_dollar_volume = dollar_volume.percentile_between(99, 100)
     
    pipe = Pipeline(
        screen = high_dollar_volume,
        columns = {
            'dollar_volume': dollar_volume
        }
    )
    return pipe
 
def before_trading_start(context, data):
    """
    Called every day before market open.
    """
    context.output = pipeline_output('my_pipeline')
  
    # These are the securities that we are interested in trading each day.
    context.security_list = context.output.index
     
def my_assign_weights(context, data):
    """
    Assign weights to securities that we want to order.
    """
    pass
 
def my_rebalance(context,data):
    """
    Execute orders according to our schedule_function() timing. 
    """
    pass
 
def my_record_vars(context, data):
    """
    Plot variables at the end of each day.
    """
    pass
 
def handle_data(context,data):
    """
    Called every minute.
    """
    pass

In [ ]:
def initialize(context):
    context.aapl = sid(24)

For now, we're just going to define our Apple stock. If you actually begin to type out sid(, Quantopian has a nice auto completion functionality where you can begin to either type the company's name or ticker symbol to find their sid. The reason for using sid is because company tickers can change over periods of time. This is one way to ensure that you're getting the ticker you're actually intending to get. You can also use symbol() to use the ticker, and make your code a bit more easy to read, but this is not recommended, since the ticker can change.

The initialize method runs once upon the starting of the algorithm (or once a day if you are running the algorithm live in real time). Handle_data runs once per minute period.

Within our initialize method, we pass this context parameter. Context is a Python Dictionary, which is what we'll use to track what we might otherwise use global variables for. Put simply, the context variable is used to track our current investment situation, with things like our portfolio and cash.

Next, we still need our handle_data function. This function takes both context and data as parameters.

The context parameter has already been explained, and the data variable is used to track the environment outside of our actual portfolio. This tracks things like stock prices and other information about companies that we may be invested in, or not, but they're companies we're tracking.

In [ ]:
def handle_data(context,data):
    # prices for aapl for the last 50 days, in 1 day intervals
    hist = data.history(context.aapl,'price', 50, '1d')
    # mean of the entire 200 day history
    sma_50 = hist.mean()
    # mean of just the last 50 days
    sma_20 = hist[-20:].mean()